In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
#data_path = os.path.join('..', 'Data', 'seq_struct_data.csv')
#df = pd.read_csv(data_path)
data_path = os.path.join('..', 'Data', 'seq_data.csv')
df = pd.read_csv(data_path)
#df.columns

In [2]:
def getData(data):
    data = data.reindex(np.random.permutation(df.index))
    data = data.values
    X = data[:,:-2]
    y = data[:,-1]
    #Change the y to a 2dimensionnal 
    ytemp = []
    for i in y:
        if i is 0:
            ytemp.append([1,0])
        else:
            ytemp.append([0,1])
    y = np.array(ytemp)
    
    t = 0.15 #keep 15 percent for testing
    x_train = X[int(t*len(X)):]
    x_test = X[:int(t*len(X))]
    y_train = y[int(t*len(y)):]
    y_test = y[:int(t*len(y))]
    return (x_train,y_train,x_test,y_test)
    
xy = getData(df)

In [3]:
#2 is hard coded but it should be the number of unique things in y or xy[1]
numbClasses = 2
numbDim= len(xy[0][0])

# Parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100
display_step = 1

#Network Parameters
n_hidden_1 = 260
n_hidden_2 = 260
n_input = numbDim 
n_classes = numbClasses 

# Create model
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.matmul(_X, _weights['h1']) + _biases['b1']) #Hidden layer with RELU activation
    layer_2 = tf.nn.relu(tf.matmul(layer_1, _weights['h2']) + _biases['b2']) #Hidden layer with RELU activation
    return tf.matmul(layer_2, weights['out']) + biases['out']

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = multilayer_perceptron(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



In [6]:
# Train
def Train():
    xy = getData(df)
    init = tf.initialize_all_variables()

    sess = tf.InteractiveSession()
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = len(xy[0])/batch_size
        for i in range(total_batch):
            batch = [xy[0][i:i+batch_size],xy[1][i:i+batch_size]]
            sess.run(optimizer, feed_dict={x: batch[0], y: batch[1]})
            avg_cost += sess.run(cost, feed_dict={x: batch[0], y: batch[1]})/total_batch
    #     if epoch % display_step == 0:
    #        print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)
    # Test trained model

    correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    return sess.run(accuracy,feed_dict={x: xy[2], y: xy[3]})

In [7]:
n = 100
results = []
for i in range(n):
    results+=[Train()]

In [9]:
# %matplotlib inline
import seaborn as sns
sns.set_context("talk")
sns.distplot(results)
#sns.plt.show()
path_out = os.path.join('..', 'Results', 'res.pdf')
sns.plt.savefig(path_out)